In [116]:
import gc
 
# Returns the number of
# objects it has collected
# and deallocated
collected = gc.collect()
   
# Prints Garbage collector 
# as 0 object
print("Garbage collector: collected",
          "%d objects." % collected)

Garbage collector: collected 371 objects.


In [137]:
import numpy as np
import pandas as pd
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor

color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

In [119]:
path = "C:\\Users\\regangala\\Documents\\GitHub\\Data-Science\\Zomato Challenge\\"
train = pd.read_csv(path + "Eatery_TrainingData.csv")
test = pd.read_csv(path + "Eatery_TestData.csv")

In [120]:
train['dataset'] = "train"
test['dataset'] = "test"
data = pd.concat([train,test], axis = 0)

In [121]:
# Data Dictory creation with Python and replacing the values 
UserRating = pd.read_csv(path + "UserRatingTrans.csv",encoding='ISO-8859-1')
UserRating_dict = {row[0]: row[1] for row in UserRating.values}
data.user_rating__rating_text=data.user_rating__rating_text.map(UserRating_dict)
data.user_rating__rating_text.value_counts()

Good         9547
Very Good    7024
Average      6524
Not rated    5149
Excellent    1515
Poor          197
Name: user_rating__rating_text, dtype: int64

In [122]:
# URL and #N/A converted to 1 and 0 respectivily. 
data['book_url'] = data['book_url'].where(data['book_url'].isnull(), 1).fillna(0).astype(int)
data['featured_image'] = data['featured_image'].where(data['featured_image'].isnull(), 1).fillna(0).astype(int)
data['zomato_events'] = data['zomato_events'].where(data['zomato_events'].isnull(), 1).fillna(0).astype(int)
data['user_rating__custom_rating_text'] = data['user_rating__custom_rating_text'].where(data['user_rating__custom_rating_text'].isnull(), 1).fillna(0).astype(int)
data['is_book_form_web_view']=data['is_book_form_web_view'].fillna(1)
data['medio_provider']=data['medio_provider'].fillna(0)
data['is_zomato_book_res']=data['is_zomato_book_res'].fillna(0)
data['mezzo_provider']=data['mezzo_provider'].fillna('NA')

data['location__country_id'] = data['location__country_id'].apply(str)
#data['establishment_types__establishment_type__id'] = data['establishment_types__establishment_type__id'].apply(str)


In [123]:
data[data['currency'].isnull()]

,id,average_cost_for_two,book_url,cuisines,currency,establishment_types__establishment_type__id,establishment_types__establishment_type__name,featured_image,has_online_delivery,has_table_booking,include_bogo_offers,is_book_form_web_view,is_delivering_now,is_table_reservation_supported,is_zomato_book_res,location__address,location__city,location__city_id,location__country_id,location__latitude,location__locality,location__locality_verbose,location__longitude,location__zipcode,medio_provider,mezzo_provider,name,offers,opentable_support,price_range,switch_to_order_menu,thumb,user_rating__custom_rating_text,user_rating__custom_rating_text_background,user_rating__rating_text,user_rating__rating_tool_tip,user_rating__votes,zomato_events,Zomato_user_rating,dataset
594,19443,450,0,Chinese,NaN,16,Casual Dining,0,0,0,True,0.0,0,0,0.0,"E 1/3, Main Road, Malvia Nagar, New Delhi, AAA...",Dummy,72,17,28.534194,AAAA,"AAAA, Dummy",77.209342,NaN,0.0,OTHER,Subway,[],0.0,4,0,NaN,0,NaN,Not rated,NaN,0,0,0.0,train
628,19466,100,0,Seafood,NaN,18,Fine Dining,0,0,0,True,0.0,0,0,0.0,"Shop 9, The Berg Centre, Corner Gordon Road & ...",Dummy,72,17,73.990000,AAAA,"AAAA, Dummy",28.880000,10000,0.0,OTHER,Fishaways,[],0.0,4,0,NaN,0,NaN,Not rated,NaN,0,0,0.0,train
1121,19492,10000,0,Japanese,NaN,21,Quick Bites,0,0,0,True,0.0,0,0,0.0,"Avenida Sur, 1001, Maipú, Santiago",Dummy,72,17,-33.521332,AAAA,"AAAA, Dummy",-70.770832,NaN,0.0,OTHER,Sushi Maipu,[],0.0,4,0,NaN,0,NaN,Not rated,NaN,0,0,0.0,train
1361,19406,15,0,Cafe,NaN,1,Café,0,0,0,True,0.0,0,0,0.0,"12 Montagu Terraco, Leith, Edinburgh, AAAA, Dummy",Dummy,72,17,73.990000,AAAA,"AAAA, Dummy",28.880000,NaN,0.0,OTHER,Cafe Montagu,[],0.0,1,0,NaN,0,NaN,Not rated,NaN,0,0,0.0,train
1468,19484,200,0,Healthy Food,NaN,21,Quick Bites,0,0,0,True,0.0,0,0,0.0,"Regent Road, Near Clarens Road, Sea Point, Cap...",Dummy,72,17,28.552751,Sea Point,"Sea Point, Dummy",77.194603,NaN,0.0,OTHER,Kauai,[],0.0,4,0,NaN,0,NaN,Not rated,NaN,0,0,0.0,train
1777,19473,50000,0,Others,NaN,18,Fine Dining,0,0,0,True,0.0,0,0,0.0,"Avenida Ossa, 2234, La Reina, Santiago",Dummy,72,17,-33.432818,AAAA,"AAAA, Dummy",-70.574542,NaN,0.0,OTHER,Eladio,[],0.0,4,0,NaN,0,NaN,Not rated,NaN,0,0,0.0,train
2038,19430,550,0,"North Indian, Chinese, South Indian",NaN,16,Casual Dining,0,0,0,True,0.0,0,0,0.0,"Western Express Highway, Near Laxmi Baug, Godb...",Dummy,72,17,73.990000,AAAA,"AAAA, Dummy",28.880000,NaN,0.0,OTHER,Hotel Sai Palace Garden,[],0.0,4,0,NaN,0,NaN,Not rated,NaN,0,0,0.0,train
2066,19416,0,0,Cafe,NaN,1,Café,0,0,0,True,0.0,0,0,0.0,"85, Brunswick Street, Fitzroy, ABAB, Dummy 3065",Dummy,72,17,-37.804622,ABAB,"ABAB, Dummy",144.977106,3065,0.0,OTHER,Martha Ray's,[],0.0,1,0,NaN,0,NaN,Not rated,NaN,0,0,0.0,train
2434,19439,1000,0,Indian,NaN,16,Casual Dining,0,0,0,True,0.0,0,0,0.0,"C 7, Main Market, Malviya Nagar, New Delhi",Dummy,72,17,28.536456,AAAA,"AAAA, Dummy",77.211625,NaN,0.0,OTHER,Moti Mahal Delux,[],0.0,4,0,NaN,0,NaN,Not rated,NaN,0,0,0.0,train
3023,19469,8700,0,Japanese,NaN,18,Fine Dining,0,0,0,True,0.0,0,0,0.0,"Providencia, 2222, Providencia, Santiago",Dummy,72,17,-33.421784,AAAA,"AAAA, Dummy",-70.609834,NaN,0.0,OTHER,Niu Sushi,[],0.0,4,0,NaN,0,NaN,Not rated,NaN,0,0,0.0,train


In [124]:
#Currency Convertion
currency = pd.read_excel(path + "currency.xlsx")
data=data.merge(currency, how='left')
data['Avg_price_per_two']=data['average_cost_for_two']*data['Value']

In [127]:
data['establishment_types__establishment_type__name'].value_counts()

Quick Bites        4055
Café               3980
Casual Dining      3687
Dessert Parlour    2744
Bar                2598
Bakery             2371
Fine Dining        2066
Pub                1825
Food Court         1516
Lounge             1015
Kiosk               940
Beverage Shop       884
Club                830
Wine Bar            334
Sweet Shop          309
Food Truck          277
Sandwich Shop       155
Bistro              115
Beer Garden         100
Cocktail Bar         87
Dhaba                75
Pop up               58
Microbrewery         12
Name: establishment_types__establishment_type__name, dtype: int64

In [128]:
#Column separtation
RemoveColumns = ['establishment_types__establishment_type__id','currency','Value','average_cost_for_two','name','include_bogo_offers','opentable_support','switch_to_order_menu','thumb','user_rating__rating_tool_tip','offers','user_rating__custom_rating_text_background']
LocationColumns = ['location__address','location__city','location__city_id','location__latitude','location__locality','location__locality_verbose','location__longitude','location__zipcode']
BinaryColumns = ['has_online_delivery','has_table_booking','is_book_form_web_view','is_delivering_now','is_table_reservation_supported','is_zomato_book_res']
DummiesColumns = ['currency_B','establishment_types__establishment_type__name','mezzo_provider','user_rating__rating_text']

In [129]:
#Remove columns not required
data = data[data.columns.difference(RemoveColumns)]
data = data[data.columns.difference(LocationColumns)]

In [130]:
cuisines_one = data.cuisines.str.get_dummies(sep = ",")
data = pd.concat([data,cuisines_one], axis=1).drop('cuisines', axis = 1)
data = pd.get_dummies(data, columns = DummiesColumns)

In [131]:
data.head()

,Avg_price_per_two,Zomato_user_rating,book_url,dataset,featured_image,has_online_delivery,has_table_booking,id,is_book_form_web_view,is_delivering_now,is_table_reservation_supported,is_zomato_book_res,location__country_id,medio_provider,price_range,user_rating__custom_rating_text,user_rating__votes,zomato_events,Aceh,Aegean,Afghan,Afghani,African,American,Andhra,Arabian,Argentine,Armenian,Asian,Asian Fusion,Australian,Austrian,Author,Awadhi,BBQ,Baiana,Bakery,Bakmi,Bakso,Balkans,Balti,Bangladeshi,Bar Food,Belanda,Belgian,Bengali,Betawi,Beverages,Bihari,Biryani,Black Sea,Brazilian,Breakfast,British,Bubble Tea,Burger,Burmese,Börek,Cafe,Cafe Food,Cafeteria,Caipira,Cake,Cambodian,Canadian,Cantonese,Cape Malay,Capixaba,Caribbean,Carvery,Central Anatolia,Chettinad,Chilean,Chinese,Coffee,Coffee and Tea,Colombian,Contemporary,Continental,Creole,Crepes,Cuban,Curry,Czech,Deli,Desserts,Dim Sum,Dimsum,Diner,Drinks Only,Dumplings,Durban,Döner,Eastern Anatolia,Eastern European,Egyptian,Emirati,Empanadas,Ethiopian,European,Falafel,Fast Food,Filipino,Finger Food,Fish and Chips,Fondue,Foul,French,Fresh Fish,Fried Chicken,Frozen Yogurt,Fusion,Gaúcha,Georgian,German,Giblets,Goan,Gourmet Fast Food,Greek,Grill,Gujarati,Hainanese,Hakka Chinese,Hawaiian,Healthy Food,Hokkien,Home-made,Hot Pot,Hungarian,Hyderabadi,Ice Cream,Ikan Bakar,Indian,Indonesian,International,Iranian,Irish,Italian,Izgara,Japanese,Jawa,Jewish,Juices,Kaak,Kebab,Kerala,Khaleeji,Kiwi,Korean,Korean BBQ,Kumpir,Latin American,Lebanese,Lounge,Madeiran,Maharashtrian,Malay,Malaysian,Manakish,Mandi,Mangalorean,Martabak,Mediterranean,Mexican,Middle Eastern,Mineira,Mishti,Mithai,Modern European,Momos,Mongolian,Moroccan,Mughlai,Nepalese,Nikkei,North Eastern,North Indian,Old Turkish Bars,Oriental,Others,Ottoman,Pacific,Padang,Pakistani,Pan Asian,Pastry,Patisserie,Peranakan,Peruvian,Petiscos,Pho,Pizza,Polish,Portuguese,Pub Food,Qatari,Raclette,Rajasthani,Ramen,Restaurant Cafe,Roast,Roast Chicken,Rolls,Russian,Saj,Salad,Sandwich,Satay,Scandinavian,Scottish,Seafood,Shawarma,Sichuan,Singaporean,Snack Bar,Snacks,Soto,South African,South East,South Indian,Southern,Spanish,Sri Lankan,Steak,Steamboat,Street Food,Sunda,Sushi,Swedish,Swiss,Syrian,Taiwanese,Tapas,Tea,Teochew,Teppanyaki,Tex-Mex,Thai,Tibetan,Turkish,Turkish Pizza,Vegan,Vegetarian,Venezuelan,Vietnamese,Western,World Cuisine,Wraps,Yemeni,Yum Cha,Aegean,Afghan,Afghani,African,Alentejana,American,Andhra,Arabian,Argentine,Armenian,Asian,Asian Fusion,Australian,Austrian,Author,Awadhi,BBQ,Bagels,Baiana,Bakery,Bakmi,Bakso,Balkans,Balti,Bangladeshi,Bar Food,Belgian,Bengali,Betawi,Beverages,Bihari,Biryani,Black Sea,Brasserie,Brazilian,Breakfast,British,Bubble Tea,Burger,Burmese,Börek,Cafe,Cafe Food,Cafeteria,Cajun,Canadian,Cantonese,Cape Malay,Caribbean,Carvery,Central Anatolia,Chettinad,Chifa,Chilean,Chinese,Chinese Muslim,Coffee,Coffee and Tea,Colombian,Contemporary,Continental,Crepes,Cuban,Czech,Deli,Desserts,Dim Sum,Dimsum,Diner,Drinks Only,Dumplings,Durban,Döner,Eastern Anatolia,Egyptian,Emirati,Empanadas,Ethiopian,European,Fast Food,Filipino,Finger Food,Fish and Chips,Fondue,French,Fresh Fish,Fried Chicken,Frozen Yogurt,Fusion,Georgian,German,Giblets,Goan,Gourmet Fast Food,Greek,Grill,Gujarati,Hainanese,Hakka Chinese,Hawaiian,Healthy Food,Hokkien,Home-made,Hong Kong Style,Hyderabadi,Ice Cream,Indian,Indonesian,International,Iranian,Irish,Italian,Izgara,Jamaican,Japanese,Jawa,Jewish,Juices,Kaak,Kebab,Kerala,Khaleeji,Kiwi,Korean,Kumpir,Latin American,Lebanese,Lithuanian,Lucknowi,Madeiran,Maharashtrian,Malay,Malaysian,Manado,Manakish,Mandi,Mangalorean,Mauritian,Mediterranean,Mexican,Middle Eastern,Mineira,Minhota,Mithai,Modern European,Modern Indian,Mongolian,Moroccan,Mozambican,Mughlai,Nepalese,Nikkei,North Eastern,North Indian,Northern Chinese,Old Turkish Bars,Oriental,Others,Ottoman,Paan,Pacific,Pakistani,Palembang,Pastry,Patisserie,Peranakan,Peruvian,Petiscos,Pilav,Pizza,Polish,Portuguese,Pub Food,Raclette,Rajasthani,Ramen,Raw Meats,Restauran

In [93]:
#Corelation of numericals
corr_matrix = data.corr()
corr_matrix["Zomato_user_rating"].sort_values(ascending=False)

In [132]:
# Checking Missing values # Target will get missing values 
all_data_na = (data.isnull().sum() / len(data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

,Missing Ratio
Zomato_user_rating,20.001332
Avg_price_per_two,0.316319


In [133]:
numerics = ['uint8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
ProcessedDataTrain = data[data.dataset == "train"] \
    .select_dtypes(include=numerics) \
    .fillna(0) 
    
ProcessedDataTest = data[data.dataset == "test"] \
    .select_dtypes(include=numerics) \
    .fillna(0) 

In [134]:
ProcessedDataTrain.to_csv('ProcessedDataTrain.csv', index = False)
ProcessedDataTest.to_csv('ProcessedDataTest.csv', index = False)

### Data Modelling 

In [135]:
# Select only numeric data and impute missing values as 0
numerics = ['uint8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
train_x = data[data.dataset == "train"] \
    .select_dtypes(include=numerics) \
    .drop("Zomato_user_rating", axis = 1) \
    .fillna(0) \
    .values

test_x = data[data.dataset == "test"] \
    .select_dtypes(include=numerics) \
    .drop("Zomato_user_rating", axis = 1) \
    .fillna(0) \
    .values
    
train_y = data[data.dataset == "train"].Zomato_user_rating.values

### ------------------------Blocking Start Random Forest ------------------------------------- 

In [111]:
from sklearn.model_selection import KFold
cv_groups = KFold(n_splits=10)
regr = RandomForestRegressor(random_state = 3, n_estimators = 500, criterion='mse',n_jobs=-1 )

In [112]:
for train_index, test_index in cv_groups.split(train_x):
    
    # Train the model using the training sets
    regr.fit(train_x[train_index], train_y[train_index])
    
    # Make predictions using the testing set
    pred_rf = regr.predict(train_x[test_index])
    
    # Calculate RMSE for current cross-validation split
    rmse = str(np.sqrt(np.mean((train_y[test_index] - pred_rf)**2)))
    
    print("RMSE for current split: " + rmse)

RMSE for current split: 0.155887177850075
RMSE for current split: 0.1490673796042891
RMSE for current split: 0.15167677852450157
RMSE for current split: 0.15167158390185087
RMSE for current split: 0.1523315011486376
RMSE for current split: 0.14617099159837565
RMSE for current split: 0.14630505798390492
RMSE for current split: 0.15030248823076742
RMSE for current split: 0.14808991586425754
RMSE for current split: 0.1536749517358234


In [113]:
# Create submission file
regr.fit(train_x, train_y)
final_prediction = regr.predict(test_x)

In [114]:
submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','Zomato_user_rating'])
submission.to_csv("Zomato_V1.csv", index = False)

### ----------------Blocking End --------------------

In [138]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(train_x, train_y, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [ ]:
# make predictions
final_prediction = my_model.predict(test_x)

submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','Zomato_user_rating'])
submission.to_csv("Zomato_V2.csv", index = False)

#from sklearn.metrics import mean_absolute_error
#print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))

In [ ]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.001)
my_model.fit(train_x, train_y, early_stopping_rounds=5, 
             eval_set=[(test_x, test_y)], verbose=False)

In [136]:

from sklearn.ensemble import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV


ImportError: cannot import name 'XGBRegressor'

In [103]:
clf = xgb()

In [105]:
# this here is the base classifier we are going to try
# we will be supplying different parameter ranges to our randomSearchCV which in turn
# will pass it on to this classifier

# Utility function to report best scores. This simply accepts grid scores from 
# our randomSearchCV/GridSearchCV and picks and gives top few combination according to 
# their scores

# RandomSearchCV/GridSearchCV accept parameters values as dictionaries.
# In example given below we have constructed dictionary for 
#different parameter values that we want to
# try for randomForest model

param_dist = {"n_estimators":[100,200,300,500,700,1000],
              "max_features": [5,10,20,25,30,35],
              "bootstrap": [True, False],
              'class_weight':[None,'balanced'], 
                'criterion':['entropy','gini'],
                'max_depth':[None,5,10,15,20,30,50,70],
                'min_samples_leaf':[1,2,5,10,15,20], 
                'min_samples_split':[2,5,10,15,20]
                  }



In [110]:
# run randomized search
n_iter_search = 100 #goog number would be 50 or 100 more
# n_iter parameter of RandomizedSeacrhCV controls, how many 
# parameter combination will be tried; out of all possible given values
# cv should be 10 to 15
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   scoring='roc_auc',cv=10,
                                  n_jobs=-1,verbose=20)
random_search.fit(train_x, train_y)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\regangala\AppData\Local\Continuum\anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\